In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch_geometric.utils import negative_sampling
import networkx as nx
import math
from random import choice
from functools import lru_cache
from sympy import prime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
with open('musae_ES/musae_ES_train.edgelist', 'r') as file:
    lines = file.readlines()

train_edges = []

for line in lines:
    elements = line.strip().split()
    edge = (int(elements[0]), int(elements[1]))
    train_edges.append(edge)

In [3]:
def enclosing_subgraph_nodes(G, x, y, K):
    F = {x, y}
    enc = [x, y]

    while len(enc) < K:
        new_neighbors = set()
        for node in F:
            neighbors = [v for v in list(G.neighbors(node)) if v not in F and v not in enc]
            new_neighbors.update(neighbors)

        if not new_neighbors:
            break  # Break if no new neighbors are found

        enc.extend(new_neighbors)
        F = new_neighbors

    # Trim down or add random dummy nodes as needed
    if len(enc) > K:
        enc = enc[:K]
    elif len(enc) < K:
        while len(enc) < K:
            random_dummy_node = choice([node for node in G.nodes() if node not in enc])
            enc.append(random_dummy_node)

    return enc

In [4]:
def initialize_colors(H, x, y):
    # to get initial colors, first need to calculate sqrt(d(v,x)*d(v,y)) for all v in node set
    mean_distances = {}
    for v in H.nodes():
        if nx.has_path(H, v, x) and nx.has_path(H, v, y):
            distance = math.sqrt(len(nx.shortest_path(H, v, x)) * len(nx.shortest_path(H, v, y)))
            mean_distances[v] = distance
        else:
            # Handle the case when there is no path between v and x or v and y
            mean_distances[v] = float('inf')  # or any other appropriate value

    # then need to map distances to colors s.t. only x=y=1 while all others get at least 2
    # depending on their mean distance

    initial_coloring = {x: 1, y: 1}

    ordering = sorted(set(mean_distances.values()))[1:]

    for v in H.nodes():
        if v not in {x, y}:
            initial_coloring[v] = 2 + ordering.index(mean_distances[v])

    return initial_coloring


In [5]:
@lru_cache(maxsize=None)
def hash_fun(v, H, **kwargs):
    current_color_v = kwargs[str(v)]

    all_mapping = {z: math.log(prime(kwargs[z])) for z in kwargs.keys()}
    neighborhood_mapping = {z: all_mapping[str(z)] for z in H.neighbors(v)}

    sum_all_mapping = sum(all_mapping.values())
    sum_neighborhood_mapping = sum(neighborhood_mapping.values())

    return current_color_v + sum_neighborhood_mapping / math.ceil(sum_all_mapping)

In [6]:
# single iteration of palette_wl, returns colors (doesn't mutate H labels)
def refine(color_dict, H):
    hashes = {v:hash_fun(v, H, **{str(k): v for k, v in color_dict.items()}) for v in color_dict.keys()}

    coloring = {}

    ordering = sorted(set(hashes.values()))

    for v in color_dict.keys():
      coloring[v] = ordering.index(hashes[v]) + 1

    return coloring

In [7]:
# 'color' attributes of H are NOT mutated by this fn either
def palette_wl(color_dict, H):
  num_iter = 0
  
  while True:
    new_color_dict = refine(color_dict, H)

    # convergence condition
    if color_dict == new_color_dict:
      break
    else:
      color_dict = new_color_dict
      num_iter += 1

  return color_dict
    
  # print(f'Converged in {num_iter} iterations')

In [8]:
def break_ties(color_dict):
    reversed_mapping = {}  # to keep track of nodes with the same color
    new_color = max(color_dict.values()) + 1  # start assigning new colors

    for node, color in color_dict.items():
        if color in reversed_mapping:
            # node with the same color already encountered
            # assign a new color and update the mapping
            color_dict[node] = new_color
            new_color += 1
        else:
            reversed_mapping[color] = node

    return color_dict

In [9]:
def process_graph(edge, H):
    x,y = edge
    initial_color_dict = initialize_colors(H, x, y)
    refined_color_dict = palette_wl(initial_color_dict, H)
    color_dict_no_ties = break_ties(refined_color_dict)
    
    sorted_nodes = sorted(color_dict_no_ties, key=color_dict_no_ties.get)
    
    return nx.to_numpy_array(H, nodelist=sorted_nodes)

In [10]:
def flatten_matrix(matrix, exclude_indices): # exclude (1,0) and (0,1) which encode existence of link
    flattened_matrix = []

    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix[i])):
            if (i, j) not in exclude_indices and (j, i) not in exclude_indices:
                flattened_matrix.append(matrix[i][j])

    return flattened_matrix

In [11]:
G = nx.from_edgelist(train_edges)
G.remove_edges_from(nx.selfloop_edges(G))

extracted_subgraphs = {}

for (i,j) in train_edges:
    if i != j: # ignore self loops
        enclo_nodes = enclosing_subgraph_nodes(G, i, j, 10)
        H = G.subgraph(enclo_nodes)
        extracted_subgraphs[(i,j)] = H

In [12]:
transformed_adj_matrices = {}

for edge, H in tqdm(extracted_subgraphs.items()):
    transformed_adj_matrices[edge] = process_graph(edge,H)

100%|██████████| 41692/41692 [21:24<00:00, 32.46it/s]  


Generate synthetic negative edges (edges not in edges_train) and repeat procedure for them (enclo. subgraph -> sorted adj. matrix). 
These are used only for training the classifier.

In [13]:
negative_edges = negative_sampling(torch.as_tensor(np.transpose(train_edges)), force_undirected=False, num_neg_samples=2*len(train_edges))

negative_edges_list = []

for i in range(len(negative_edges[0])):
    negative_edges_list.append((int(negative_edges[0][i]),int(negative_edges[1][i])))

G_fake = nx.from_edgelist(train_edges)
G_fake.add_edges_from(negative_edges_list)
G_fake.remove_edges_from(nx.selfloop_edges(G_fake))

extracted_subgraphs_fake = {}

for (i,j) in negative_edges_list:
    if i != j: # ignore self loops
        enclo_nodes = enclosing_subgraph_nodes(G_fake, i, j, 10)
        H_fake = G_fake.subgraph(enclo_nodes)
        extracted_subgraphs_fake[(i,j)] = H_fake

transformed_adj_matrices_fake = {}

for edge_fake, H_fake in tqdm(extracted_subgraphs_fake.items()):
    transformed_adj_matrices_fake[edge_fake] = process_graph(edge_fake,H_fake)

100%|██████████| 83384/83384 [09:19<00:00, 149.04it/s]


In [14]:
cnt=0
for i,j in negative_edges_list:
    if G.has_edge(i,j) or G.has_edge(j,i):
        cnt+=1 
cnt

164

Prepare training data

In [15]:
train_data = []

for matrix in transformed_adj_matrices.values():
    flattened_matrix = flatten_matrix(matrix, {(0, 1), (1, 0)})
    train_data.append({'label':1, 'features': flattened_matrix})


for matrix in transformed_adj_matrices_fake.values():
    flattened_matrix = flatten_matrix(matrix, {(0, 1), (1, 0)})
    train_data.append({'label':0, 'features': flattened_matrix})  
        

train_df = pd.DataFrame(train_data)
train_df_shuffled = train_df.iloc[np.random.permutation(len(train_df))].reset_index(drop=True)

In [16]:
X = np.vstack(train_df_shuffled['features'].to_numpy())
y = train_df_shuffled['label'].to_numpy()

classifier = LogisticRegression(max_iter=100, solver='liblinear')

classifier.fit(X,y)

LogisticRegression(solver='liblinear')

Extract enclosing subgraphs for test edges and compute sorted adjacency matrices 

In [17]:
with open('musae_ES/musae_ES_test.edgelist', 'r') as file:
    lines = file.readlines()

test_edges = []

for line in lines:
    elements = line.strip().split()
    edge = (int(elements[0]), int(elements[1]))
    test_edges.append(edge)

G_test = nx.from_edgelist(train_edges)
G_test.add_edges_from(test_edges)
G_test.remove_edges_from(nx.selfloop_edges(G_test))

extracted_test_subgraphs = {}

for (i,j) in test_edges:
    if i != j: # ignore self loops
        enclo_nodes = enclosing_subgraph_nodes(G_test, i, j, 10)
        H_test = G_test.subgraph(enclo_nodes)
        extracted_test_subgraphs[(i,j)] = H_test

transformed_adj_matrices_test = {}

for edge_test, H_test in tqdm(extracted_test_subgraphs.items()):
    transformed_adj_matrices_test[edge_test] = process_graph(edge_test,H_test)

100%|██████████| 17690/17690 [12:27<00:00, 23.68it/s] 


Generate synthetic negative samples for testing (edges not in test_edges OR train_edges) and do same procedure for them as above

In [18]:
# make sure negative samples are truly negative 
train_union_test = train_edges + test_edges
negative_test_edges = negative_sampling(torch.as_tensor(np.transpose(train_union_test)), force_undirected=False, num_neg_samples=2*len(test_edges))

negative_test_edges_list = []

for i in range(len(negative_test_edges[0])):
    negative_test_edges_list.append((int(negative_test_edges[0][i]),int(negative_test_edges[1][i])))
    

G_fake_test = nx.from_edgelist(train_edges)
G_fake_test.add_edges_from(negative_test_edges_list)
G_fake_test.remove_edges_from(nx.selfloop_edges(G_fake_test))

extracted_subgraphs_fake_test = {}

for (i,j) in negative_test_edges_list:
    if i != j: # ignore self loops
        enclo_nodes = enclosing_subgraph_nodes(G_fake_test, i, j, 10)
        H_fake_test = G_fake_test.subgraph(enclo_nodes)
        extracted_subgraphs_fake_test[(i,j)] = H_fake_test

transformed_adj_matrices_fake_test = {}

for edge_fake_test, H_fake_test in tqdm(extracted_subgraphs_fake_test.items()):
    transformed_adj_matrices_fake_test[edge_fake_test] = process_graph(edge_fake_test,H_fake_test)

100%|██████████| 35380/35380 [06:42<00:00, 87.91it/s] 


In [19]:
cnt=0
for i,j in negative_test_edges_list:
    if G_test.has_edge(i,j) or G_test.has_edge(j,i):
        cnt+=1 
cnt

93

Prepare test data

In [20]:
test_data = []

for matrix in transformed_adj_matrices_test.values():
    flattened_matrix = flatten_matrix(matrix, {(0, 1), (1, 0)})
    test_data.append({'label':1, 'features': flattened_matrix})

for matrix in transformed_adj_matrices_fake_test.values():
    flattened_matrix = flatten_matrix(matrix, {(0, 1), (1, 0)})
    test_data.append({'label':0, 'features': flattened_matrix})  

test_df = pd.DataFrame(test_data)
test_df_shuffled = test_df.iloc[np.random.permutation(len(test_df))].reset_index(drop=True)
print(test_df_shuffled)

       label                                           features
0          1  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
1          0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2          0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3          0  [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
4          0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
...      ...                                                ...
53065      0  [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
53066      0  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
53067      0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
53068      0  [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...
53069      1  [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...

[53070 rows x 2 columns]


In [21]:
X_test = np.vstack(test_df_shuffled['features'].to_numpy())
y_test = test_df_shuffled['label'].to_numpy()

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.10f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.8046542303
              precision    recall  f1-score   support

           0       0.86      0.85      0.85     35380
           1       0.70      0.72      0.71     17690

    accuracy                           0.80     53070
   macro avg       0.78      0.78      0.78     53070
weighted avg       0.81      0.80      0.81     53070


In [22]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, classifier.predict_proba(X_test)[:, 1])

0.8552546278617265